## Run KNN on dataset using sql

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

### Quality Measures
Aroma, Flavor, Aftertaste, Acidity, Body, Balance, Uniformity, Cup Cleanliness, Sweetness, Moisture, Defects  

### Bean Metadata
Processing Method, Color, Species (arabica / robusta)  

### Farm Metadata
Owner, Country of Origin, Farm Name, Lot Number, Mill, Company, Altitude, Region

In [2]:
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import connect
import json 
from sqlalchemy import create_engine

/usr/local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [3]:
# load json file in dir for db details

with open('config.json') as f:
    conf = json.load(f)

In [4]:
# use fstring to fill in items from the json document
login = f"postgresql://{conf['user']}:{conf['passw']}@{conf['host']}/coffee" 

In [5]:
conn = create_engine(login).connect() # connect and fill db from csv file
df = pd.read_csv('/Users/sumac/projects/metis/project3/arabica_data_cleaned.csv') 
df.to_sql('arabica2', conn)

OperationalError: (psycopg2.OperationalError) could not translate host name "covfefe-metis.c4dhyd2spzjx.us-west-1.rds.amazonaws.com" to address: nodename nor servname provided, or not known
 (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
drop_columns = ['index','Species','Unnamed: 0', 'Owner','Farm.Name','Lot.Number', 'Mill', 'ICO.Number', 'Company', \
                         'Altitude', 'Region', 'Producer','Bag.Weight', 'In.Country.Partner','Harvest.Year', 'Grading.Date', \
                         'Owner.1','Variety','Color','Quakers','Expiration','Number.of.Bags', 'Certification.Body','Certification.Address', \
                         'Certification.Contact', 'unit_of_measurement']

column_names = ['Country.of.Origin', 'Processing.Method', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', \
                         'Body', 'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points', \
                         'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 'Category.Two.Defects', \
                         'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters']

In [ ]:
drop_template = """ALTER TABLE arabica2
                    DROP COLUMN "{column_name}";"""

In [ ]:
for drop_col in drop_columns: # loop to drop unneeded cols
    drop_statement = drop_template.format(column_name=drop_col) 
    conn.execute(drop_statement)

In [ ]:
df = pd.read_sql("SELECT * FROM arabica2", con = conn)
df.head()

In [6]:
rename_col_template = """ALTER TABLE arabica2 RENAME COLUMN 
                        "{column_name}" TO "{new_column_name}";"""

In [ ]:
for col in column_names: # loop to rename cols
    name_statement = rename_col_template.format(column_name=col, new_column_name=col.lower().replace('.','_'))
    conn.execute(name_statement)

In [22]:
df = pd.read_sql("SELECT * FROM arabica2", con = conn) # check to make sure things worked
df.head()

,country_of_origin,processing_method,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,cupper_points,total_cup_points,moisture,category_one_defects,category_two_defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,Ethiopia,Washed / Wet,8.67,8.83,8.67,8.75,8.50,8.42,10.0,10.0,10.0,8.75,90.58,0.12,0,0,1950.0,2200.0,2075.0
1,Ethiopia,Washed / Wet,8.75,8.67,8.50,8.58,8.42,8.42,10.0,10.0,10.0,8.58,89.92,0.12,0,1,1950.0,2200.0,2075.0
2,Guatemala,None,8.42,8.50,8.42,8.42,8.33,8.42,10.0,10.0,10.0,9.25,89.75,0.00,0,0,1600.0,1800.0,1700.0
3,Ethiopia,Natural / Dry,8.17,8.58,8.42,8.42,8.50,8.25,10.0,10.0,10.0,8.67,89.00,0.11,0,2,1800.0,2200.0,2000.0
4,Ethiopia,Washed / Wet,8.25,8.50,8.25,8.50,8.42,8.33,10.0,10.0,10.0,8.58,88.83,0.12,0,2,1950.0,2200.0,2075.0


In [47]:
df['processing_method'].isna().sum()

152

In [27]:
data = df.dropna() # can figure out how to do this drop in sql later

In [28]:
data.dtypes

country_of_origin        object
processing_method        object
aroma                   float64
flavor                  float64
aftertaste              float64
acidity                 float64
body                    float64
balance                 float64
uniformity              float64
clean_cup               float64
sweetness               float64
cupper_points           float64
total_cup_points        float64
moisture                float64
category_one_defects      int64
category_two_defects      int64
altitude_low_meters     float64
altitude_high_meters    float64
altitude_mean_meters    float64
dtype: object

In [57]:
dummy = pd.get_dummies(data, columns=['processing_method'], drop_first=True) # creating dummies in sql - possible?
dummy.head()

,country_of_origin,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,...,moisture,category_one_defects,category_two_defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters,processing_method_Other,processing_method_Pulped natural / honey,processing_method_Semi-washed / Semi-pulped,processing_method_Washed / Wet
0,Ethiopia,8.67,8.83,8.67,8.75,8.50,8.42,10.0,10.0,10.0,...,0.12,0,0,1950.0,2200.0,2075.0,0,0,0,1
1,Ethiopia,8.75,8.67,8.50,8.58,8.42,8.42,10.0,10.0,10.0,...,0.12,0,1,1950.0,2200.0,2075.0,0,0,0,1
3,Ethiopia,8.17,8.58,8.42,8.42,8.50,8.25,10.0,10.0,10.0,...,0.11,0,2,1800.0,2200.0,2000.0,0,0,0,0
4,Ethiopia,8.25,8.50,8.25,8.50,8.42,8.33,10.0,10.0,10.0,...,0.12,0,2,1950.0,2200.0,2075.0,0,0,0,1
9,Ethiopia,8.08,8.58,8.50,8.50,7.67,8.42,10.0,10.0,10.0,...,0.10,0,4,1795.0,1850.0,1822.5,0,0,0,0


In [9]:
#add region column
regions = { 'United States': 'north_america',
           'Mexico': 'north_america',
           'China' : 'asia',
           'Taiwan': 'asia',
           'Japan':'asia',
           'El Salvador': 'central_america_carib',
           'Nicaragua': 'central_america_carib',
           'Costa Rica': 'central_america_carib',
           'Guatemala':'central_america_carib',
           'Honduras':'central_america_carib',
           'Haiti':'central_america_carib',
           'United States (Puerto Rico)':'central_america_carib',
           'Brazil' : 'south_america',
           'Peru': 'south_america',
           'Colombia':  'south_america',
           'Panama': 'south_america',
           'Ecuador': 'south_america',
           'Ethiopia':'africa',
           'Kenya': 'africa',
           'Burundi':'africa',
           'Cote d?Ivoire':'africa',
           'Rwanda':'africa',
           'Malawi':'africa',
           'Uganda':'africa',
           'Tanzania, United Republic Of': 'africa',
           'Zambia': 'africa',
           'Thailand':'southeast_asia',
           'Myanmar':'southeast_asia',
           'Indonesia':'southeast_asia',
           'Papua New Guinea':'southeast_asia',
           'Vietnam':'southeast_asia',
           'Philippines':'southeast_asia',
           'Laos':'southeast_asia'
          }

def get_region(search_country): 
    for (country, region) in regions.items():
            if country == search_country:
                return region

In [10]:
region_list=[]

for row in dummy['country_of_origin']:
    print(get_region(row))
    region_list.append(get_region(row))
    
dummy['region'] = region_list

africa
africa
africa
africa
africa
africa
north_america
north_america
north_america
asia
africa
north_america
central_america_carib
north_america
north_america
south_america
africa
asia
south_america
central_america_carib
africa
africa
north_america
southeast_asia
asia
africa
south_america
south_america
africa
africa
africa
south_america
south_america
africa
central_america_carib
central_america_carib
southeast_asia
central_america_carib
central_america_carib
south_america
africa
africa
africa
central_america_carib
africa
africa
south_america
asia
central_america_carib
central_america_carib
central_america_carib
africa
africa
south_america
south_america
southeast_asia
central_america_carib
south_america
africa
south_america
africa
central_america_carib
north_america
africa
africa
south_america
south_america
asia
africa
south_america
north_america
south_america
south_america
asia
africa
africa
central_america_carib
central_america_carib
southeast_asia
asia
central_america_carib
central_

In [11]:
dummy.shape

(1005, 23)

In [12]:
dummy.to_csv('arabica_to_classify.csv') # export and check the csv
csv_check = pd.DataFrame.from_csv('arabica_to_classify.csv')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [13]:
csv_check = pd.DataFrame.from_csv('arabica_to_classify.csv')

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [14]:
csv_check.head()

,country_of_origin,aroma,flavor,aftertaste,acidity,body,balance,uniformity,clean_cup,sweetness,...,category_one_defects,category_two_defects,altitude_low_meters,altitude_high_meters,altitude_mean_meters,processing_method_Other,processing_method_Pulped natural / honey,processing_method_Semi-washed / Semi-pulped,processing_method_Washed / Wet,region
0,Ethiopia,8.67,8.83,8.67,8.75,8.50,8.42,10.0,10.0,10.0,...,0,0,1950.0,2200.0,2075.0,0,0,0,1,africa
1,Ethiopia,8.75,8.67,8.50,8.58,8.42,8.42,10.0,10.0,10.0,...,0,1,1950.0,2200.0,2075.0,0,0,0,1,africa
3,Ethiopia,8.17,8.58,8.42,8.42,8.50,8.25,10.0,10.0,10.0,...,0,2,1800.0,2200.0,2000.0,0,0,0,0,africa
4,Ethiopia,8.25,8.50,8.25,8.50,8.42,8.33,10.0,10.0,10.0,...,0,2,1950.0,2200.0,2075.0,0,0,0,1,africa
9,Ethiopia,8.08,8.58,8.50,8.50,7.67,8.42,10.0,10.0,10.0,...,0,4,1795.0,1850.0,1822.5,0,0,0,0,africa


In [15]:
csv_check.shape

(1005, 23)

In [ ]:
#where did the unnamed col come from??? using read_csv breaks the csv